In [1]:
import pandas as pd
import plotly.plotly as py
%load_ext sql
%sql sqlite://
%config SqlMagic.feedback = False #to turn off sqlite reports

In [42]:
## Load csv files into SQL via pandas #
 # All raw data save to table ALLPOST ##

location_list=["Toronto_ON", "Montreal_QC", "Vancouver_BC", "Waterloo_ON", "New-York_NY", "Boston_MA", "Washington_DC", 
               "Seattle_WA", "San-Francisco_CA", "Austin_TX", "San-Jose_CA", "Raleigh_NC", "Houston_TX", "Portland_OR", 
               "Atlanta_GA", "San-Diego_CA", "Denver_CO", "Minneapolis_MN", "Saint-Louis_MO"]
%sql DROP TABLE IF EXISTS ALLPOST
%sql CREATE TABLE ALLPOST (keyword varchar, location varchar, link varchar, post_text varchar)
for location in location_list:
    df = pd.read_csv("D:\monster_scrapingdata\softdev_" + location + "_20170116.csv", sep='\t', header=0, index_col=0)
    %sql DROP TABLE IF EXISTS df
    %sql PERSIST df
    %sql INSERT INTO ALLPOST SELECT keyword, location, link, post_text FROM df 

In [58]:
techlist=["javascript","sql", 'java', 'c#', 'php', 'c++', 'c', 'node.js', 'angularjs', 'ruby', 'rails', 'python',
          'delphi', 'typescript', 'bash', 'groovy', 'powershell', 'html', 'css', 'django','postgresql', 'jquery', 'ios', 'git', 
          'mysql', '.net', 'arrays', 'swift', 'objective-c', 'string', 'json', 'sql server', 'linux', 'regex', 'asp.net', 
         'react', 'spark', 'cassandra', 'raspberry pi', 'wordpress', 'mongodb', 'redis', 'vba', 'hadoop', 'clojure',"visual basic",
          'f#', 'sharepoint', 'rust', 'scala', 'cordova', 'perl', 'salesforce', 'matlab', 'dart', 'coffeescript',
         'haskell', 'windows phone', 'aws', 'gae', 'azure', 'lamp', "lua", "drupal", "elixir", "fortran", "erlang", "pl/sql", 
         "elasticsearch", "coldfusion", "kotlin", "smalltalk", "abap", "elm", "r", "cloud", "sas", "d3", "spss",
         "excel", "tableau", "pig", "mapreduce", "hive", "shark", "oozie", "zookeeper", "flume", "mahout", "android"]

In [33]:
## Complimentary Skills for each location ##
df_to = pd.read_csv("D:\monster_scrapingdata\softdev_Toronto_ON_20170116.csv", sep='\t', header=0, index_col=0)
%sql DROP TABLE IF EXISTS df_to
%sql PERSIST df_to

'Persisted df_to'

In [59]:
from itertools import permutations
allperms = list(permutations(techlist, 2))

In [68]:
l=[]
for index, (skill1, skill2) in enumerate(allperms):
    l.append((skill1, skill2, df_to[df_to["post_text"].str.contains(",%s," % skill1, regex = False) | df_to["post_text"].str.contains(",%s," % skill2, regex = False)].shape[0]))
    if index % 1000 == 0:
        print("-")

-
-
-
-
-
-
-
-
-


In [69]:
comp_skills = pd.DataFrame(l, columns = ["skill", "complementary_skill", "all_posting"])

In [70]:
comp_skills.head()

,skill,complementary_skill,all_posting
0,javascript,sql,403
1,javascript,java,523
2,javascript,c#,261
3,javascript,php,273
4,javascript,c++,316


In [74]:
comp_skills.sort_values("all_posting", ascending= False).groupby("skill").head(5).sort_values(["skill", "all_posting"], ascending = [True, False])

,skill,complementary_skill,all_posting
2227,.net,java,522
2299,.net,cloud,386
2225,.net,javascript,372
2226,.net,sql,351
2231,.net,c,330
6410,abap,java,400
6408,abap,javascript,262
6414,abap,c,258
6409,abap,sql,248
6482,abap,cloud,226


In [44]:
# ranking skills#
def ranking(techlist, location, citycount):
    comskill=[]
    for tech in techlist:
        acount=%sql SELECT Count(*) FROM ALLPOST WHERE location = :location AND (post_text LIKE "%,$tech,%" OR post_text LIKE "$tech,%" OR post_text LIKE "%,$tech")
        comskill.append((location, tech, acount[0][0], acount[0][0]/citycount)) 
    return sorted(comskill, key=lambda x: x[2], reverse= True)

In [45]:
complist=[]
for location in location_list:
    techlistcp=techlist
    citycount=%sql SELECT Count(*) FROM ALLPOST WHERE location = :location
    for i in range(5):
        coms=ranking(techlistcp, location, citycount[0][0]) # call ranking func #
        complist.append(coms[0])
        ele= coms[0][1]
        techlistcp.remove(ele)
        %sql delete from ALLPOST WHERE location = :location AND post_text LIKE "%,$ele,%"
    print ("done" + location) 

doneToronto_ON
doneMontreal_QC
doneVancouver_BC
doneWaterloo_ON
doneNew-York_NY
doneBoston_MA
doneWashington_DC
doneSeattle_WA
doneSan-Francisco_CA
doneAustin_TX
doneSan-Jose_CA
doneRaleigh_NC
doneHouston_TX
donePortland_OR
doneAtlanta_GA
doneSan-Diego_CA
doneDenver_CO
doneMinneapolis_MN


IndexError: list index out of range

In [46]:
complist

[('Toronto_ON', 'java', 399, 0.399),
 ('Toronto_ON', 'c', 181, 0.181),
 ('Toronto_ON', 'javascript', 75, 0.075),
 ('Toronto_ON', 'cloud', 54, 0.054),
 ('Toronto_ON', 'excel', 48, 0.048),
 ('Montreal_QC', '.net', 346, 0.346),
 ('Montreal_QC', 'sql', 150, 0.15),
 ('Montreal_QC', 'linux', 117, 0.117),
 ('Montreal_QC', 'c++', 76, 0.076),
 ('Montreal_QC', 'html', 41, 0.041),
 ('Vancouver_BC', 'ruby', 181, 0.181),
 ('Vancouver_BC', 'python', 105, 0.105),
 ('Vancouver_BC', 'css', 65, 0.065),
 ('Vancouver_BC', 'r', 52, 0.052),
 ('Vancouver_BC', 'asp.net', 26, 0.026),
 ('Waterloo_ON', 'android', 91, 0.15716753022452504),
 ('Waterloo_ON', 'git', 32, 0.055267702936096716),
 ('Waterloo_ON', 'jquery', 21, 0.03626943005181347),
 ('Waterloo_ON', 'php', 16, 0.027633851468048358),
 ('Waterloo_ON', 'hadoop', 14, 0.024179620034542316),
 ('New-York_NY', 'aws', 76, 0.07778915046059365),
 ('New-York_NY', 'mysql', 43, 0.044012282497441144),
 ('New-York_NY', 'ios', 33, 0.033776867963152504),
 ('New-York_NY', 